AdaBoost 属于**集成学习**算法的一种。集成学习通过构建多个学习任务。结构框架如图所示

![这里写图片描述](http://img.blog.csdn.net/20180117224333866?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvcXFfMjY5MTE2MTk=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)

集成学习通过多个分类器进行结合，因而大多数情况下拥有较好的泛化性能，以AdaBoost为例，其集成方法各有千秋：可以是同一算法在不同设置下集成；也可以是在不同数据集上集成，一般数据进行抽样训练。
不同的学习器要有一定的准确性，又要有差异性。
如图是Ada的模型：

![这里写图片描述](http://img.blog.csdn.net/20180117224830436?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvcXFfMjY5MTE2MTk=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)
右边矩形代表不同权重下的数据集（同一数据集）

AdaBoost既可以用于分类问题，也可以用于回归问题。
关于如何推导出算法的简洁公式：（二分类为例）
二分类问题 $y\in \{1,-1\}$ 和真实的取值函数$f(x)$, 假设基础分类器的错误率为$\epsilon$,对于每个分类器$h_i(x)$ 有
$$P[h_i(x)\ne f(x)]=\epsilon$$
原则上若超过半数以上的分类器分类正确，则认为集成分类就是正确的
集成分类器是其线性组合：
$$H(x)=sign \bigg( \sum_{t=1}^{T}h_i(x)\biggr)$$
而在AdaBoost中，有一种加权线性模型：
$$H(x)= \sum_{t=1}^{T}\alpha_i h_i(x)$$
其中$\alpha_i$为各自分类器的权重。
在分布$D$上，（D为分类器权重向量），定义的最小化损失函数$L_{exp}$的表达式为（exp为 exponential loss function 即最小化损失函数）：
$$L_{exp}(H|D)=e^{-f(x)H(x)}$$
一个多元复合函数，要求其最小化，则求偏导；这里需要对目标函数也就是$H(x)$求偏导：
$H(x)包含取值为1的H_1和取值为-1的H_2$,因而链式求偏导过程为：
$$e^{-f(x)H(x)}:\begin{cases}
f(x):\begin{cases}1,\\-1\end{cases}\\
H(x):\begin{cases}H_1(x)\\H_2(x)\end{cases}
\end{cases}$$

那么：

$\frac{\partial L_{exp}(H|D)}{\partial H(x)}=\frac {\partial e^{-f(x)H(x)}}{\partial H(x)}=\frac {\partial e^{-f_1(x)H_1(x)}}{\partial H_1(x)}+\frac {\partial e^{-f_{-1}(x)H_{-1}(x)}}{\partial H(x)}\\=e^{f_1{(x)H_1(x)}}[-f_1(x)]{\frac {\partial H(x)}{\partial H_1(x)}}+e^{f_{-1}{(x)H_{-1}(x)}}[-f_{-1}(x)]{\frac {\partial H(x)}{\partial H_{-1}(x)}}$

因为$f_{1}(x)=1$与$f_{-1}(x)=-1$,代入上式得：

$\frac{\partial L_{exp}(H|D)}{\partial H(x)}=-e^{-H_1(x)}P(f(x)=1|x)+e^{H_{-1}(x)}P(f(x)=-1|x)$

令其为0求解：
$e^{-H_1(x)}P(f(x)=1|x)=e^{H_{-1}(x)}P(f(x)=-1|x) \\ \Rightarrow \frac{e^{H_{-1}(x)}}{e^{-H_{1}(x)}}=\frac{P(f(x)=1|x)}{P(f(x)=-1|x)}=e^{H_1(x)+H_{-1}(x)}=e^{2H(x)} \\两边取对数: \\\Rightarrow H(x)=\frac{1}{2}ln \frac{P(f(x)=1|x)}{P(f(x)=-1|x)}$

>值得注意的是 $H(x)$一开始由基于算法初始数据分布而来。

因为：$H_t(x)=\alpha_tf(x)$,那么：
$$\frac{\partial L_{exp}(\alpha_tf(x)|D_t)}{\partial \alpha_t}=\frac{1}{2}ln \frac{1-\epsilon}{\epsilon}  \Leftarrow权重更新公式$$

算法获得$H_{t-1}$后，下一轮学习的$h_t$将修正$H_{t-1}$的错误：
$$L_{exp}((H_{t-1}+h_t)|D)=e^{-f(x)[H_{t-1}(x)+h_t]}=e^{-f(x)H_{t-1}(x)}*e^{-f(x)h_t(x)}$$

对后一项进行二阶泰勒展开：
因为：二阶泰勒展开：$e^x=1-x+\frac{x^2}{2}$

故上式得：$e^{-f(x)H_{t-1}(x)}*[1-f(x)h_t(x)+\frac{f_(x)^2h_t(x)^2}{2}]$

又因为 $f_(x)^2=h_t(x)^2=1 \Rightarrow e^{-f(x)H_{t-1}(x)}*[1-f(x)h_t(x)+\frac{1}{2}]$ 

$argmax=e^{-f(x)H_{t-1}(x)}f(x)h_t(x)$

$argmax=\frac{e^{-f(x)H_{t-1}(x)}}{E_x\sim D[e^{-f(x)H_{t-1}(x)}]}f(x)h_t(x)$

那么每次权重$D$的迭代：
$$D_{t+1}=\frac{D_t(x)e^{-f(x)H_{t}(x)}}{Sum(D(x))}=\frac{D_t(x)e^{-\alpha_tf(x)h_{t}(x)}}{Sum(D(x))}$$



